### 朴素贝叶斯 鸢尾花


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

x,y = load_iris().data,load_iris().target
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=50)
len(x_test)


#训练模型
model = GaussianNB()
model.fit(x_train,y_train)

#评估
pred = model.predict(x_test)
print('预测集数据的预测标签是：',pred)
print('测试机数据的真实标签是:',y_test)
acc = accuracy_score(y_test,pred)
print('预测的准确率是：',acc)
sum(pred!=y_test)

# 垃圾邮件

In [1]:
import os 
normalFileList = os.listdir('../bayes/normal')
spamFileList = os.listdir('../bayes/spam')
print('正常邮件列表为：',normalFileList)
print('垃圾邮件列表为：',spamFileList)

In [4]:
stopList = []
for line in open('../bayes/stopwords.txt','r',encoding='utf-8'):
    stopList.append(line[:len(line) - 1])
print(stopList)

In [5]:
from jieba import cut
from re import sub

def getWord(file,stopList):
    wordslist = []
    for line in open(file,encoding='utf-8'):
        line = line.strip()
        line = sub(r'[.【】0-9、——，。！\~*]','',line)
        line = cut(line)
        line = filter(lambda word:len(word) > 1,line)
        wordslist.extend(line)
        words = []
        for i in wordslist:
            if i not in stopList and i.strip() != '' and i != None:
                words.append(i)
                
    return words
        

In [7]:
from collections import Counter
from itertools import chain

allwords = []
for normalfile in normalFileList:
    words = getWord('../bayes/normal/'+normalfile,stopList)
    allwords.append(words)
for spamfile in spamFileList:
    words = getWord('../bayes/spam/'+spamfile,stopList)
    allwords.append(words)
print('训练集中所有的有效词语列表：',allwords)

In [12]:
frep = Counter(chain(*allwords))
topTen = frep.most_common(10)
topTen
topWords = [i[0] for i in topTen]
print('训练集中出现频率最高的10个词语是：',topWords)


In [13]:
import numpy as np
vector = []
for words in allwords:
    temp = list(map(lambda x:words.count(x),topWords))
    vector.append(temp)
vector = np.array(vector)
print('10个高频词语在每封邮件中出现的次数：\n',vector)

In [14]:
from sklearn.naive_bayes import MultinomialNB

target = np.array([0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1])

x,y = vector,target
model = MultinomialNB()
model.fit(x,y)


In [15]:
test = os.listdir('../bayes/test')
for testFile in test:
    words = getWord('../bayes/test/'+testFile,stopList)
    test_x = np.array(tuple(map(lambda word:words.count(word),topWords)))
    result = model.predict(test_x.reshape(1,-1))
    if result == 1:
        print(testFile,'为垃圾邮件')
    else:
        print(testFile,'为正常邮件')